# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import sys
from sqlalchemy import create_engine
import nltk
import joblib

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as iPipeline

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score

from mismote import MLSMOTE

[nltk_data] Downloading package punkt to /home/salman1567/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/salman1567/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part I: Loading the Dataset

In [9]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster.db', engine)
# X = df.iloc[:,1]
# Y = df.iloc[:,4:]

# We can drop child alone since there aren't any messages with that category.


#print(X.head())

### Method 1: Undersampling
In this method we will only take a subset of smaples from the entire dataset. Knowing this should be okay since most of the text data have similar text.


In [10]:


def underSampleDataSet(df, samples=200):
    df_resampled = []
    df.drop(['child_alone'], axis=1, inplace=True)
    for i in df.iloc[:,4:].columns:
        new_df = df[df[i]==1]
        print(new_df.shape)
        df_resampled.append(new_df.sample(200, replace=True))
    
    # Concating the 200 samples from each category into one and returning it.
    df = pd.concat(df_resampled)
    return df

df = underSampleDataSet(df)




(20042, 39)
(4513, 39)
(121, 39)
(10954, 39)
(2099, 39)
(1319, 39)
(725, 39)
(471, 39)
(863, 39)
(1684, 39)
(2956, 39)
(2342, 39)
(410, 39)
(606, 39)
(301, 39)
(880, 39)
(1204, 39)
(3464, 39)
(1709, 39)
(1212, 39)
(1345, 39)
(538, 39)
(159, 39)
(283, 39)
(120, 39)
(309, 39)
(1155, 39)
(7343, 39)
(2177, 39)
(2464, 39)
(282, 39)
(2461, 39)
(533, 39)
(1379, 39)
(5113, 39)


In [12]:
# check if they have been resampled

print(df.shape)

(7000, 39)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
14394,17180,"Hola Hospital, used as a referral hospital by ...",None,news,1,0,0,1,1,0,...,0,0,1,1,0,0,0,0,0,0
3033,3509,I am a pastor 41 of my people have lost their ...,I am a pastor 41 of my people have lost their ...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
13110,15772,Priority interventions will focus on the provi...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7879,8850,"GOOD EVENING, I'M A NURSE WHO ISN'T WORKING AN...","Bonswa, mwen se enfimy ki pa nan travay k'ap v...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20992,24470,The project creates the opportunity for those ...,None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


This dataset is imbalanced so we will try to attempt to merge some of the really small categories which have less than 200 samples. This does mean we lose some specificity with our model and this is an important consideration if this trade off is acceptable or not. It can be argued in some situations we don't need to know the exact specific disaster but rather the general category such as if it is infrastructural related we make sure we send out a team with tools and general capability to handle that kind of situation.

We will be merging buildings, electricity, tools, hospitals, shops, aid-centers, other_infrastructure all into `infrastructure_related`. The reason for doing this makes sense from a logical point of view since those are all infrastructure related and will typically mean we send out similar teams prepared for any kind of infrastructure related problems.

We will also be placing clothing, money, missing people, refugees and death category into `other_aid`



## Part II: Experimenting with Feature Engineering

Attempting to experiment with feature engineering to see if f1 score will improve by making the model more general and merging some of the indepdent features together.

In [6]:

def merge_features(df):
      # If any of these features are 1 than infrastructure_related should be 1 since we merging them in
      
      df.loc[(df['buildings']==1) |
            (df['electricity']==1) |
            (df['tools']==1) |
            (df['hospitals']==1) |
            (df['shops']==1) |
            (df['aid_centers']==1) |
            (df['other_infrastructure']==1), 'infrastructure_related'] = 1

      # Merging categories into other aid

      df.loc[(df['clothing']==1) |
            (df['money']==1) |
            (df['missing_people']==1) |
            (df['refugees']==1) |
            (df['death']==1), 'other_aid'] = 1

      # Merging floods, storm, cold and other_weather into weather_related

      df.loc[(df['floods']==1) |
            (df['storm']==1) |
            (df['cold']==1) |
            (df['other_weather']==1), 'weather_related'] = 1

      # for i in df.columns.unique()[7:]:
      #       print(df.columns.unique())
      #       if df[df[i]==1].shape[0] > 1000:
      #             pass
      #       else:
      #             df = df.drop(labels=[i])


      # Dropping the merged in features
      df = df.drop(labels=['buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
            'clothing', 'money', 'missing_people', 'refugees', 'death', 'floods', 'storm', 'cold', 'other_weather'], axis=1)

# merge_features(df)

#### Feature merging

Later when running the pipeline we see that this merging improves f1 score very slightly, lowering precision but improving recall thus overall with an improved f1 score. Since we are dealing with an imbalanced dataset we will prioritize recall and f1 score over precision.

In [13]:
# Making sure that the features successfuly merged into infrastructure_related
X = df.iloc[:,1]
Y = df.iloc[:,4:]

for i in Y.columns.unique():
    print(i, Y[Y[i]==1].shape[0], sep=" - ")

related - 7000
request - 2127
offer - 239
aid_related - 5985
medical_help - 1533
medical_products - 1235
search_and_rescue - 836
security - 601
military - 585
water - 1104
food - 1608
shelter - 1623
clothing - 487
money - 527
missing_people - 478
refugees - 887
death - 948
other_aid - 2015
infrastructure_related - 1980
transport - 1185
buildings - 1188
electricity - 582
tools - 355
hospitals - 458
shops - 317
aid_centers - 503
other_infrastructure - 1090
weather_related - 4183
floods - 1627
storm - 1335
fire - 423
earthquake - 1224
cold - 542
other_weather - 947
direct_report - 2305


In [8]:
# Do the same for other aid
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Part III: Tokenization of text data
### 2. Write a tokenization function to process your text data

In [14]:
def tokenize(text):
    
    clean_tokens = []
    
    # lowercasing and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    tokens = word_tokenize(text)
    # Remove stop words + stem
    stemmer = PorterStemmer()
    
    
    stop_words = stopwords.words("english")
    clean_tokens = [ stemmer.stem(word) for word in tokens if word not in stop_words]

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# build a transformer that scales
class OverSamplingTransformer(BaseEstimator, TransformerMixin):
    """A custom transformer to be used in the ML pipeline to handle the class imbalance that is present
    in this dataset.
    """
    
    def __init__(self, n_sample):
        self.n_sample = n_sample
        pass
    
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        y = pd.DataFrame(y)
        return MLSMOTE(X, y, self.n_sample)
    
    def transform(self, X):
        pass

In [15]:
# Try other estimators to see if performance is better
# Implement oversampling using SVD of n components

# Number of featurs we want to reduce to so that we can then use this in oversampling transformer
def build_pipeline(n_features=33):
    pipeline = Pipeline([
    #('smt', SMOTE()),
    ('features', FeatureUnion([
        ('nltk_pipeline', Pipeline([
                ('count-vector', CountVectorizer(tokenizer=tokenize)),
                ('tf-idf', TfidfTransformer())   
        ])),
        
        
        
    ])),
     #('tf-idf', TfidfVectorizer()),
    #('SVD', TruncatedSVD(n_components=n_features)),
    #('smote', OverSamplingTransformer(n_sample=1000)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ])
    
    return pipeline

pipeline = build_pipeline()




In [12]:
# Trying a different model
def build_pipeline2():

    pipeline2 = Pipeline([
        #('smt', SMOTE()),
        ('count-vector', CountVectorizer(tokenizer=tokenize)),
        ('tf-idf', TfidfTransformer()),
        #('tf-idf', TfidfVectorizer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1))
    ])
    
    return pipeline2

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('nltk_pipeline',
                                                 Pipeline(steps=[('count-vector',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f84e31257e0>)),
                                                                 ('tf-idf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

In [14]:
print(X_train)

9979     7.0 earthquake rocks Haiti The Christian Scien...
3632     I WOULD BE HAPPY BECAUSE EVEN COROSSOL JUICE,T...
18789    Because of this, at 9:05am a safety valve on t...
21277    The state power company Tenaga Nasional Bhd ha...
15451    USAID, through the Assistance for Emergency Lo...
                               ...                        
10955    Really hope this Hurricane doesn't happen . #f...
17289    In Xai-Xai, a small city about 90 miles from M...
5192     I didn't ever get an answer to the question I ...
12172    shamsher haider mirani moza sumra nashaib janoobi
235      Send cards ( airtime, calling cards ) for Digi...
Name: message, Length: 17430, dtype: object


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
col_names = Y.columns.unique()
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))
    



                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      2310
               request       0.91      0.82      0.86       692
                 offer       1.00      0.77      0.87        71
           aid_related       0.90      0.99      0.94      1976
          medical_help       0.92      0.73      0.81       498
      medical_products       0.96      0.70      0.81       402
     search_and_rescue       0.98      0.71      0.82       297
              security       0.98      0.68      0.80       201
              military       0.97      0.56      0.71       211
                 water       0.97      0.83      0.89       368
                  food       0.96      0.89      0.92       551
               shelter       0.96      0.76      0.85       542
              clothing       0.97      0.77      0.86       160
                 money       0.98      0.62      0.76       181
        missing_people       0.99      

In [16]:

def train_pipeline2(X, Y):
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.33, random_state=1)
    pipeline2.fit(X_train2, y_train2)
    col_names = Y.columns.unique()
    y_pred2 = pipeline2.predict(X_test2)
    print(classification_report(y_test2, y_pred2, target_names=col_names))

In [17]:
# Checking what hyper parameters can be tuned
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__nltk_pipeline', 'features__nltk_pipeline__memory', 'features__nltk_pipeline__steps', 'features__nltk_pipeline__verbose', 'features__nltk_pipeline__count-vector', 'features__nltk_pipeline__tf-idf', 'features__nltk_pipeline__count-vector__analyzer', 'features__nltk_pipeline__count-vector__binary', 'features__nltk_pipeline__count-vector__decode_error', 'features__nltk_pipeline__count-vector__dtype', 'features__nltk_pipeline__count-vector__encoding', 'features__nltk_pipeline__count-vector__input', 'features__nltk_pipeline__count-vector__lowercase', 'features__nltk_pipeline__count-vector__max_df', 'features__nltk_pipeline__count-vector__max_features', 'features__nltk_pipeline__count-vector__min_df', 'features__nltk_pipeline__count-vector__ngram_range', 'features__nltk_pipeline__count-vector__preprocessor', 'features__n

`TODO`: Consider to remove child_alone since there are none of those and also consider oversampling since some of the disaster categories have a low amount of samples.

In [18]:
# we can consider dropping any columns that have zero rows associated
def drop_columns_with_low_samples():
    for i in df.columns.unique():
        if sum(df[i]) < 200:
            df = df.drop(columns=[i])
        


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {'clf__estimator__criterion': ("squared_error", 'absolute_error', 'poisson'),
              'clf__estimator__max_features': ('auto', 'sqrt', 'log2'), 
              'clf__n_jobs': [-1], 
              'clf__estimator__n_jobs': [-1]}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('nltk_pipeline',
                                                                        Pipeline(steps=[('count-vector',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f01e21c5d80>)),
                                                                                        ('tf-idf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__n_jobs': [-1], 'clf__n_jobs': [-1]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))

                        precision    recall  f1-score   support

               related       0.77      0.97      0.86      6545
               request       0.41      0.04      0.08      1520
                 offer       0.00      0.00      0.00        34
           aid_related       0.44      0.21      0.29      3582
          medical_help       0.04      0.00      0.00       672
      medical_products       0.00      0.00      0.00       441
     search_and_rescue       0.14      0.00      0.01       230
              security       0.00      0.00      0.00       155
              military       0.00      0.00      0.00       308
                 water       0.15      0.01      0.01       568
                  food       0.26      0.01      0.02       973
               shelter       0.17      0.01      0.01       775
              clothing       0.00      0.00      0.00       134
                 money       0.00      0.00      0.00       188
        missing_people       0.00      

/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packag

We notice that a lot of the scores have 0 since we have a imbalanced data set. We can try to fix this by oversampling

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Check what params gridsearchcv has selected
cv.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('features',
   FeatureUnion(transformer_list=[('nltk_pipeline',
                                   Pipeline(steps=[('count-vector',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x7f01e21c5d80>)),
                                                   ('tf-idf',
                                                    TfidfTransformer())]))])),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1))],
 'estimator__verbose': False,
 'estimator__features': FeatureUnion(transformer_list=[('nltk_pipeline',
                                 Pipeline(steps=[('count-vector',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f01e21c5d80>)),
                                                 ('tf-idf',
                                                  TfidfTransformer())]))]),
 'estimator__clf

### 9. Export your model as a pickle file

In [27]:
# Export the model as a pickle file
joblib.dump(cv, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [23]:
model = joblib.load('classifier.pkl')

In [24]:
model.predict(["".join("we are more than 50 people sleeping on the street. Please hep us find tent, food.")])


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [25]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16452,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
13143,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25308,1,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21432,1,0,0,1,0,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,0
17497,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22275,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_test[0]

'Weather update - a cold front from Cuba that could pass over Haiti'